In [7]:
# Ignore irritating pandas warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import numpy as np
import sweetviz as sv
from redcap import Project
from tableone import TableOne
wd = '/Volumes/PEDS/RI Biostatistics Core/Shared/Shared Projects/Laura/BDC/'\
    'Projects/Laurel Messer/SEEDS/'
api = pd.read_table(wd+'/Data_Raw/api_token.txt')
api_url = 'https://redcap.ucdenver.edu/api/'
project = Project(api_url, api)
df = pd.DataFrame(project.export_records(raw_or_label='label'))
df.replace('', np.nan, inplace=True)


In [8]:
# Data cleaning
# Real records only
df = df[df.real_record == 'Real']
df.loc[df['otherspecialty'] == 'Pediatric Diabetology',
       'specialty'] = 'Pediatric Endocrinology'
df.loc[df['otherspecialty'] == 'Adult Gerontology Primary Care',
       'specialty'] = 'Primary care-family medicine'
# Specialty and provider type
df['specialty'].replace(dict.fromkeys(['Primary care-family medicine', 'Primary care-internal medicine',
                                       'Primary care-pediatrics', 'Primary care-med/peds'], 'Primary Care'), inplace=True)
prescribe_replace = dict.fromkeys(['Physician (MD, DO or equivalent)',
                                  'Advance practice professional (Physician assistant, nurse practitioner)', 'Endocrinology fellow'], 'Prescriber')
prescribe_replace.update(dict.fromkeys(
    ['Diabetes care and education specialist (diabetes educator, nurse, dietitian, other)', 'Medical resident'], 'Non-Prescriber'), inplace=True)
df['providertype'].replace(prescribe_replace, inplace=True)
# Drop 'Other' providers
df = df.loc[df['specialty'] != 'Other', :]


# Table 1: Participant Characteristics


In [9]:
cat_vars = ['gender','race','providertype','specialty','practicesetting','medicare','medicaid','privateins','uninsured']
t1_vars = ['age','practiceyears','cgmpatientratio','pumppatientratio','penpatientratio','hclpatientratio','educationtime']+cat_vars
group = 'specialty'
t1 = TableOne(data=df,columns=t1_vars,categorical=cat_vars, groupby=group,dip_test=True, normal_test=True, tukey_test=True)
t1

Grouped by specialty                                                                      
                                                                 Missing      Overall Adult endocrinology Pediatric Endocrinology Primary Care
n                                                                                 244                  89                      36          115
age, mean (SD)                                                         4  44.8 (11.1)          41.8 (9.6)             42.8 (11.7)  47.7 (11.3)
practiceyears, mean (SD)                                               4  13.8 (10.3)          10.8 (9.2)             13.1 (12.1)  16.3 (10.0)
cgmpatientratio, mean (SD)                                            10  42.4 (28.8)         53.0 (21.1)             72.3 (15.4)  25.4 (25.6)
pumppatientratio, mean (SD)                                           10  20.3 (18.6)         22.1 (14.3)             42.7 (22.9)  12.2 (13.5)
penpatientratio, mean (SD)                                            10   9.9 (16.4)          6.9 (10.8)             10.9 (11.3)  11.9 (20.4)
hclpatientratio, mean (SD)                                            10  22.3 (23.1)         29.2 (21.2)             45.4 (18.4)  10.1 (17.7)
educationtime, mean (SD)                                              10  20.3 (16.3)         21.6 (17.6)             27.9 (14.2)  17.0 (15.1)
gender, n (%)               Female                                     4   150 (62.5)           65 (73.0)               29 (80.6)    56 (48.7)
                            Male                                            90 (37.5)           24 (27.0)                7 (19.4)    59 (51.3)
race, n (%)                 Asian/Pacific Islander                     4    40 (16.7)           16 (18.0)                4 (11.1)    20 (17.4)
                            Black                                             5 (2.1)                                                  5 (4.3)
                            Native American                                   1 (0.4)                                     1 (2.8)             
                            Other                                            19 (7.9)           12 (13.5)                 2 (5.6)      5 (4.3)
                            White                                          175 (72.9)           61 (68.5)               29 (80.6)    85 (73.9)
providertype, n (%)         Non-Prescriber                             4    43 (17.9)           16 (18.0)                8 (22.2)    19 (16.5)
                            Other                                             7 (2.9)             1 (1.1)                 1 (2.8)      5 (4.3)
                            Prescriber                                     190 (79.2)           72 (80.9)               27 (75.0)    91 (79.1)
specialty, n (%)            Adult endocrinology                        4    89 (37.1)          89 (100.0)                                     
                            Pediatric Endocrinology                         36 (15.0)                                  36 (100.0)             
                            Primary Care                                   115 (47.9)                                              115 (100.0)
practicesetting, n (%)      Rural                                      4    36 (15.0)             4 (4.5)                 1 (2.8)    31 (27.0)
                            Suburban                                        93 (38.8)           40 (44.9)                5 (13.9)    48 (41.7)
                            Urban                                          111 (46.2)           45 (50.6)               30 (83.3)    36 (31.3)
medicare, n (%)             0%                                         6    29 (12.2)             2 (2.3)               24 (66.7)      3 (2.6)
                            1-25%                                           81 (34.0)           24 (27.6)               11 (30.6)    46 (40.0)
                            26-50%              

# Exploratory data analyis

In [14]:
compare=sv.compare_intra(df,df['providertype']=='Prescriber',['Prescriber','Non-Prescriber'])
#compare.show_html(filepath=wd+'Reports/seeds_eda.html')

Feature: care_type_v2                        |▏         | [  2%]   00:00 -> (00:01 left)